## 1. Import package

In [1]:
# necessary package
from fractions import Fraction
import pandas as pd
import numpy as np

## 2. drop off columns

In [2]:
df = pd.read_csv('restaurantfinalmerge.csv')

In [4]:
df = pd.read_csv('restaurantfinalmerge.csv')
df = df.drop('Unnamed: 0', axis = 1)
df = df.rename(columns={'Unnamed: 0.1': 'menu'})
df = df.reindex(['brand', 'menu', 'calories', 'calfromfat', 'fat', 'satfat', 'transfat', 'cho',
      'sodium', 'charbo', 'dietfac', 'sugar', 'protein'], axis='columns')
df.head()

,brand,menu,calories,calfromfat,fat,satfat,transfat,cho,sodium,charbo,dietfac,sugar,protein
0,7-Eleven,5 React Fruit Slurpee,66,?,?,?,?,0mg,6mg,18g,0g,18g,0g
1,7-Eleven,Black Iced Coffee,5,?,?,?,?,0mg,25mg,1g,0g,?,0g
2,7-Eleven,Blue Lightning Blast Slurpee,70,?,?,?,?,0mg,20mg,18g,0g,18g,0g
3,7-Eleven,Cherry Orange Blitz Slurpee,110,?,?,?,?,0mg,25mg,28g,0g,27g,0g
4,7-Eleven,Coca Cola Classic Slurpee,179,?,?,?,?,0mg,16mg,49g,0g,49g,0g


In [5]:
# drop unknown columns
df = df.drop(columns = ['satfat', 'transfat', 'dietfac', 'calfromfat'])
df.head()

,brand,menu,calories,fat,cho,sodium,charbo,sugar,protein
0,7-Eleven,5 React Fruit Slurpee,66,?,0mg,6mg,18g,18g,0g
1,7-Eleven,Black Iced Coffee,5,?,0mg,25mg,1g,?,0g
2,7-Eleven,Blue Lightning Blast Slurpee,70,?,0mg,20mg,18g,18g,0g
3,7-Eleven,Cherry Orange Blitz Slurpee,110,?,0mg,25mg,28g,27g,0g
4,7-Eleven,Coca Cola Classic Slurpee,179,?,0mg,16mg,49g,49g,0g


In [9]:
# replace all ? with NaN values and remove whitespace
df2 = df.apply(lambda x: x.str.strip())
df2 = df2.replace('?', np.NaN)

In [10]:
# How to deal with space? -> I will drop it off
df2 = df2.drop(labels=df2[df2.calories == ''].index, axis=0).reset_index()
df2 = df2.drop(columns = ['index'])

## 3. Change unit of each nutrition column

In [11]:
# calories with "g" or "mg" units
# covert these units to kcal
# Since one gram of body fat is equal to 7.716179 calories, you can use this simple formula to convert (https://www.inchcalculator.com/convert/gram-to-calorie-burned/)*
# calories = grams × 7.716179
for i, v in enumerate(df2.calories):
    if type(v) == str:
        if v[-2:] == 'mg':
            df2.loc[i, 'calories'] = float(v[:-2])/1000*7.716179
        elif v[-1] == 'g':
            df2.loc[i, 'calories'] = float(v[:-1])*7.716179

In [12]:
df2['calories'].astype(float)

0         66.0
1          5.0
2         70.0
3        110.0
4        179.0
         ...  
16409    590.0
16410    320.0
16411    780.0
16412    610.0
16413    350.0
Name: calories, Length: 16414, dtype: float64

In [13]:
# calories is done
df3 = df2
df3.head()

,brand,menu,calories,fat,cho,sodium,charbo,sugar,protein
0,7-Eleven,5 React Fruit Slurpee,66,NaN,0mg,6mg,18g,18g,0g
1,7-Eleven,Black Iced Coffee,5,NaN,0mg,25mg,1g,NaN,0g
2,7-Eleven,Blue Lightning Blast Slurpee,70,NaN,0mg,20mg,18g,18g,0g
3,7-Eleven,Cherry Orange Blitz Slurpee,110,NaN,0mg,25mg,28g,27g,0g
4,7-Eleven,Coca Cola Classic Slurpee,179,NaN,0mg,16mg,49g,49g,0g


In [14]:
# fat delete weired values
for i, v in enumerate(df3.fat):
    if type(v) == str:
        if v[-2:] == 'mg' or v[-2:] == '.g':
            df3.loc[i, 'fat'] = v[:-2]
        elif v[-1] == 'g':
            df3.loc[i, 'fat'] = v[:-1]
        elif len(v) >=5:
            if v[-5:] == 'grams':
                df3.loc[i, 'fat'] = v[:-5]
        elif '/' in v:
            df3.loc[i, 'fat'] =float(Fraction(v))

In [15]:
df3.fat = df3.fat.apply(lambda x: x.strip() if type(x) == str else x)

In [16]:
for i, v in enumerate(df3.fat):
    if type(v) == str:
        if v.replace('.','').isdigit():
            df3.loc[i, 'fat'] = float(v)

In [17]:
for i, v in enumerate(df3.fat):
    if type(v) == str:
        if v.isdecimal() == False:
            df3.drop(labels= i, axis=0, inplace = True)
        
df3 = df3.reset_index().drop(columns = ['index'])

In [18]:
# fat is done
df4 = df3
df4.head()

,brand,menu,calories,fat,cho,sodium,charbo,sugar,protein
0,7-Eleven,5 React Fruit Slurpee,66,NaN,0mg,6mg,18g,18g,0g
1,7-Eleven,Black Iced Coffee,5,NaN,0mg,25mg,1g,NaN,0g
2,7-Eleven,Blue Lightning Blast Slurpee,70,NaN,0mg,20mg,18g,18g,0g
3,7-Eleven,Cherry Orange Blitz Slurpee,110,NaN,0mg,25mg,28g,27g,0g
4,7-Eleven,Coca Cola Classic Slurpee,179,NaN,0mg,16mg,49g,49g,0g


In [19]:
df4.cho.shape

(15793,)

In [20]:
# cho delete weired values
check = ['.', '/', ' ']
for i, v in enumerate(df4.cho):
    for c in check:
        if type(v) == str:
            if v[-2:] == 'mg':
                if v[:-2].replace(c,'').isdigit() == True:
                    df4.loc[i, 'cho'] = float(v[:-2])
            elif v[-1] == 'g':
                if v[:-1].replace(c,'').isdigit() == True:
                    df4.loc[i, 'cho'] = float(v[:-1])*1000
            elif '/' in v:
                if v.replace(c,'').isdigit():
                    df4.loc[i, 'cho'] =float(Fraction(v))
            if len(v) >=5:
                if v[-5:] == 'grams':
                    if v[:-5].replace(c,'').isdigit() == True:
                        df4.loc[i, 'cho'] = float(v[:-5])*1000

for i, v in enumerate(df4.cho):
    if type(v) == str:
        if v.isdecimal() == False:
            df4.drop(labels= i, axis=0, inplace = True)
        
df4 = df4.reset_index().drop(columns = ['index'])

In [21]:
# cho is done
df5 = df4
df5.head()

,brand,menu,calories,fat,cho,sodium,charbo,sugar,protein
0,7-Eleven,5 React Fruit Slurpee,66,NaN,0,6mg,18g,18g,0g
1,7-Eleven,Black Iced Coffee,5,NaN,0,25mg,1g,NaN,0g
2,7-Eleven,Blue Lightning Blast Slurpee,70,NaN,0,20mg,18g,18g,0g
3,7-Eleven,Cherry Orange Blitz Slurpee,110,NaN,0,25mg,28g,27g,0g
4,7-Eleven,Coca Cola Classic Slurpee,179,NaN,0,16mg,49g,49g,0g


In [22]:
# sodium delete weired values
check = ['.', '/', ' ']
for i, v in enumerate(df5.sodium):
    for c in check:
        if type(v) == str:
            if v[-2:] == 'mg':
                if v[:-2].replace(c,'').isdigit() == True:
                    df5.loc[i, 'sodium'] = float(v[:-2])
            elif v[-1] == 'g':
                if v[:-1].replace(c,'').isdigit() == True:
                    df5.loc[i, 'sodium'] = float(v[:-1])*1000
            elif '/' in v:
                if v.replace(c,'').isdigit():
                    df5.loc[i, 'sodium'] =float(Fraction(v))
            if len(v) >=5:
                if v[-5:] == 'grams':
                    if v[:-5].replace(c,'').isdigit() == True:
                        df5.loc[i, 'sodium'] = float(v[:-5])*1000

# for i, v in enumerate(df5.sodium):
#     if type(v) == str:
#         if v.isdecimal() == False:
#             print(i, v)
                        
                        
for i, v in enumerate(df5.sodium):
    if type(v) == str:
        if v.isdecimal() == False:
            df5.drop(labels= i, axis=0, inplace = True)
        
df5 = df5.reset_index().drop(columns = ['index'])

In [23]:
# sodium is done
df6 = df5
df6.head()

,brand,menu,calories,fat,cho,sodium,charbo,sugar,protein
0,7-Eleven,5 React Fruit Slurpee,66,NaN,0,6,18g,18g,0g
1,7-Eleven,Black Iced Coffee,5,NaN,0,25,1g,NaN,0g
2,7-Eleven,Blue Lightning Blast Slurpee,70,NaN,0,20,18g,18g,0g
3,7-Eleven,Cherry Orange Blitz Slurpee,110,NaN,0,25,28g,27g,0g
4,7-Eleven,Coca Cola Classic Slurpee,179,NaN,0,16,49g,49g,0g


In [24]:
# charbo delete weired values
check = ['.', '/', ' ']
for i, v in enumerate(df6.charbo):
    for c in check:
        if type(v) == str:
            if v[-2:] == 'mg':
                if v[:-2].replace(c,'').isdigit() == True:
                    df6.loc[i, 'charbo'] = float(v[:-2])/1000
            elif v[-1] == 'g':
                if v[:-1].replace(c,'').isdigit() == True:
                    df6.loc[i, 'charbo'] = float(v[:-1])
            elif '/' in v:
                if v.replace(c,'').isdigit():
                    df6.loc[i, 'charbo'] =float(Fraction(v))
            if len(v) >=5:
                if v[-5:] == 'grams':
                    if v[:-5].replace(c,'').isdigit() == True:
                        df6.loc[i, 'charbo'] = float(v[:-5])

# for i, v in enumerate(df6.charbo):
#     if type(v) == str:
#         if v.isdecimal() == False:
#             print(i, v)
                        
                        
for i, v in enumerate(df6.charbo):
    if type(v) == str:
        if v.isdecimal() == False:
            df6.drop(labels= i, axis=0, inplace = True)
        
df6 = df6.reset_index().drop(columns = ['index'])

In [25]:
# charbo is done
df7 = df6
df7.head()

,brand,menu,calories,fat,cho,sodium,charbo,sugar,protein
0,7-Eleven,5 React Fruit Slurpee,66,NaN,0,6,18,18g,0g
1,7-Eleven,Black Iced Coffee,5,NaN,0,25,1,NaN,0g
2,7-Eleven,Blue Lightning Blast Slurpee,70,NaN,0,20,18,18g,0g
3,7-Eleven,Cherry Orange Blitz Slurpee,110,NaN,0,25,28,27g,0g
4,7-Eleven,Coca Cola Classic Slurpee,179,NaN,0,16,49,49g,0g


In [26]:
# sugar delete weired values
check = ['.', '/', ' ']
for i, v in enumerate(df7.sugar):
    for c in check:
        if type(v) == str:
            if v[-2:] == 'mg':
                if v[:-2].replace(c,'').isdigit() == True:
                    df7.loc[i, 'sugar'] = float(v[:-2])/1000
            elif v[-1] == 'g':
                if v[:-1].replace(c,'').isdigit() == True:
                    df7.loc[i, 'sugar'] = float(v[:-1])
            elif '/' in v:
                if v.replace(c,'').isdigit():
                    df7.loc[i, 'sugar'] =float(Fraction(v))
            if len(v) >=5:
                if v[-5:] == 'grams':
                    if v[:-5].replace(c,'').isdigit() == True:
                        df7.loc[i, 'sugar'] = float(v[:-5])

# for i, v in enumerate(df7.charbo):
#     if type(v) == str:
#         if v.isdecimal() == False:
#             print(i, v)
                        
                        
for i, v in enumerate(df7.sugar):
    if type(v) == str:
        if v.isdecimal() == False:
            df7.drop(labels= i, axis=0, inplace = True)
        
df7 = df7.reset_index().drop(columns = ['index'])

In [27]:
# sugar is done
df8 = df7
df8.head()

,brand,menu,calories,fat,cho,sodium,charbo,sugar,protein
0,7-Eleven,5 React Fruit Slurpee,66,NaN,0,6,18,18,0g
1,7-Eleven,Black Iced Coffee,5,NaN,0,25,1,NaN,0g
2,7-Eleven,Blue Lightning Blast Slurpee,70,NaN,0,20,18,18,0g
3,7-Eleven,Cherry Orange Blitz Slurpee,110,NaN,0,25,28,27,0g
4,7-Eleven,Coca Cola Classic Slurpee,179,NaN,0,16,49,49,0g


In [28]:
# protein delete weired values
check = ['.', '/', ' ']
for i, v in enumerate(df8.protein):
    for c in check:
        if type(v) == str:
            if v[-2:] == 'mg':
                if v[:-2].replace(c,'').isdigit() == True:
                    df8.loc[i, 'protein'] = float(v[:-2])/1000
            elif v[-1] == 'g':
                if v[:-1].replace(c,'').isdigit() == True:
                    df8.loc[i, 'protein'] = float(v[:-1])
            elif '/' in v:
                if v.replace(c,'').isdigit():
                    df8.loc[i, 'protein'] =float(Fraction(v))
            if len(v) >=5:
                if v[-5:] == 'grams':
                    if v[:-5].replace(c,'').isdigit() == True:
                        df8.loc[i, 'protein'] = float(v[:-5])

# for i, v in enumerate(df8.protein):
#     if type(v) == str:
#         if v.isdecimal() == False:
#             print(i, v)
                        
                        
for i, v in enumerate(df8.protein):
    if type(v) == str:
        if v.isdecimal() == False:
            df8.drop(labels= i, axis=0, inplace = True)
        
df8 = df8.reset_index().drop(columns = ['index'])

In [29]:
df8.brand.isnull()

0        False
1        False
2        False
3        False
4        False
         ...  
15781    False
15782    False
15783    False
15784    False
15785    False
Name: brand, Length: 15786, dtype: bool

## 4. Drop null values from brand and menu

In [33]:
#drop null brand
df8 = df8[df8['brand'].notna()]

In [34]:
#drop menu
df8 = df8[df8['brand'].notna()]

In [35]:
#Final
df8.shape

(15712, 9)

In [36]:
df8.to_csv('TentativeCleanedRestaurant.csv', index = False)

## 5. Remove outliers

In [51]:
data = pd.read_csv('TentativeCleanedRestaurant.csv')

In [52]:
data

,brand,menu,calories,fat,cho,sodium,charbo,sugar,protein
0,7-Eleven,5 React Fruit Slurpee,66.0,NaN,0.0,6.0,18.0,18.0,0.0
1,7-Eleven,Black Iced Coffee,5.0,NaN,0.0,25.0,1.0,NaN,0.0
2,7-Eleven,Blue Lightning Blast Slurpee,70.0,NaN,0.0,20.0,18.0,18.0,0.0
3,7-Eleven,Cherry Orange Blitz Slurpee,110.0,NaN,0.0,25.0,28.0,27.0,0.0
4,7-Eleven,Coca Cola Classic Slurpee,179.0,NaN,0.0,16.0,49.0,49.0,0.0
...,...,...,...,...,...,...,...,...,...
15707,Zoes Kitchen,Live Med Salad,590.0,34.0,5.0,1220.0,61.0,7.0,19.0
15708,Zoes Kitchen,Quinoa Salad,320.0,14.0,5.0,850.0,42.0,7.0,10.0
15709,Zoes Kitchen,Tossed Greek Salad,780.0,33.0,160.0,2540.0,58.0,10.0,64.0
15710,Zoes Kitchen,Hummus Trio,610.0,33.0,0.0,1160.0,65.0,7.0,18.0


In [53]:
# There are outliers for each nutrition column
data.describe()

,calories,fat,cho,sodium,charbo,sugar,protein
count,15707.000000,15443.000000,1.518200e+04,15706.000000,15711.000000,15482.000000,15650.000000
mean,353.928249,20.443699,3.454441e+03,689.362537,38.713035,31.172749,21.680064
std,420.001143,36.551631,3.774094e+04,920.421793,65.361432,157.554597,74.672791
min,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,101.000000,2.000000,0.000000e+00,105.000000,5.000000,1.000000,1.000000
50%,230.000000,11.000000,2.000000e+01,362.000000,24.000000,5.000000,6.000000
75%,490.000000,26.000000,7.500000e+01,960.000000,50.000000,15.000000,21.000000
max,24264.000000,1616.000000,1.247000e+06,24970.000000,2525.000000,3890.000000,1540.000000


In [54]:
limit = pd.DataFrame(columns = data.columns[2:])
for j in [0.981, 0.982, 0.983, 0.984, 0.985, 0.986, 0.987, 0.988, 0.989, 0.99]:
    for i in data.columns[2:]:
        limit.loc[j, i] = data.loc[:, i].quantile(j)
        #print(i, j, data.loc[:, i].quantile(j))

In [55]:
# Decided to go for 990, 990, 988, 990, 990, 980, 987 for each column as a cut-off
limit

,calories,fat,cho,sodium,charbo,sugar,protein
0.981,1400,114,510,3216.05,190,289.861,89.669
0.982,1410,119,535,3260,195,400,94
0.983,1430,123,589.615,3310,207,430,100
0.984,1450,127,636.832,3357.2,219.28,450,111.232
0.985,1470,132,678.99,3434.25,230,490,139
0.986,1490,140,752.33,3490,246.06,530,199.14
0.987,1520,142,805,3550,259.77,617.47,265.815
0.988,1550,142,933.28,3610,270,700,320
0.989,1580,142,134063,3652.45,280,790,380
0.990,1600,142,227000,3719.5,290,810,410


In [56]:
data.calories.quantile(0.990)

1600.0

In [57]:
data[data.columns[2]].quantile(0.990)

1600.0

In [58]:
data[data.columns[2]]

0         66.0
1          5.0
2         70.0
3        110.0
4        179.0
         ...  
15707    590.0
15708    320.0
15709    780.0
15710    610.0
15711    350.0
Name: calories, Length: 15712, dtype: float64

In [59]:
data = pd.read_csv('TentativeCleanedRestaurant.csv')
limit_list = [data[data.columns[2]].quantile(0.990), 
                  data[data.columns[3]].quantile(0.990),
                    data[data.columns[4]].quantile(0.988),
                     data[data.columns[5]].quantile(0.990),
                      data[data.columns[6]].quantile(0.990),
                        data[data.columns[7]].quantile(0.980),
                         data[data.columns[8]].quantile(0.987)]

data = data.drop(data[data.calories > limit_list[0]].index)
data = data.drop(data[data.fat > limit_list[1]].index)
data = data.drop(data[data.cho > limit_list[2]].index)
data = data.drop(data[data.sodium > limit_list[3]].index)
data = data.drop(data[data.charbo > limit_list[4]].index)
data = data.drop(data[data.sugar > limit_list[5]].index)
data = data.drop(data[data.protein > limit_list[6]].index)

In [62]:
# clean data without outliers
data

,brand,menu,calories,fat,cho,sodium,charbo,sugar,protein
0,7-Eleven,5 React Fruit Slurpee,66.0,NaN,0.0,6.0,18.0,18.0,0.0
1,7-Eleven,Black Iced Coffee,5.0,NaN,0.0,25.0,1.0,NaN,0.0
2,7-Eleven,Blue Lightning Blast Slurpee,70.0,NaN,0.0,20.0,18.0,18.0,0.0
3,7-Eleven,Cherry Orange Blitz Slurpee,110.0,NaN,0.0,25.0,28.0,27.0,0.0
4,7-Eleven,Coca Cola Classic Slurpee,179.0,NaN,0.0,16.0,49.0,49.0,0.0
...,...,...,...,...,...,...,...,...,...
15707,Zoes Kitchen,Live Med Salad,590.0,34.0,5.0,1220.0,61.0,7.0,19.0
15708,Zoes Kitchen,Quinoa Salad,320.0,14.0,5.0,850.0,42.0,7.0,10.0
15709,Zoes Kitchen,Tossed Greek Salad,780.0,33.0,160.0,2540.0,58.0,10.0,64.0
15710,Zoes Kitchen,Hummus Trio,610.0,33.0,0.0,1160.0,65.0,7.0,18.0


## 6. Add a null value dataframe for dash

In [63]:
df_nan = pd.DataFrame(columns = data.columns)
df_nan.brand = data.brand.unique()
df_nan[data.columns[2:]] = df_nan[data.columns[2:]].fillna(0)
df_nan = df_nan.append(pd.Series([np.nan for i in range(len(df_nan.columns))], index=df_nan.columns), ignore_index = True)

In [64]:
df_nan[data.columns[1]] = df_nan[data.columns[1]].replace(np.NaN, 'No choice')

In [65]:
df_nan.loc[153, 'brand'] = 'No choice'

In [66]:
df_nan

,brand,menu,calories,fat,cho,sodium,charbo,sugar,protein
0,7-Eleven,No choice,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Applebee's,No choice,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Arby's,No choice,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Au Bon Pain,No choice,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Auntie Anne's,No choice,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
149,Wingstop,No choice,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150,Yard House,No choice,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151,Zaxby's,No choice,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152,Zoes Kitchen,No choice,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
data_comp = pd.concat([data, df_nan], ignore_index=True)

## 7. Complete data

In [71]:
# complete data
data_comp

,brand,menu,calories,fat,cho,sodium,charbo,sugar,protein
0,7-Eleven,5 React Fruit Slurpee,66.0,NaN,0.0,6.0,18.0,18.0,0.0
1,7-Eleven,Black Iced Coffee,5.0,NaN,0.0,25.0,1.0,NaN,0.0
2,7-Eleven,Blue Lightning Blast Slurpee,70.0,NaN,0.0,20.0,18.0,18.0,0.0
3,7-Eleven,Cherry Orange Blitz Slurpee,110.0,NaN,0.0,25.0,28.0,27.0,0.0
4,7-Eleven,Coca Cola Classic Slurpee,179.0,NaN,0.0,16.0,49.0,49.0,0.0
...,...,...,...,...,...,...,...,...,...
15263,Wingstop,No choice,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15264,Yard House,No choice,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15265,Zaxby's,No choice,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15266,Zoes Kitchen,No choice,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
# EDA
data_comp.describe()

,calories,fat,cho,sodium,charbo,sugar,protein
count,15262.000000,15134.000000,14875.000000,15261.000000,15266.000000,15037.000000,15205.000000
mean,325.666361,16.463292,51.967800,623.557434,31.206243,12.058955,12.935942
std,313.766848,19.712593,89.455556,725.409159,32.739002,20.034218,17.016829
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,100.000000,1.000000,0.000000,90.000000,5.000000,1.000000,1.000000
50%,220.000000,10.000000,15.000000,340.000000,22.000000,4.000000,6.000000
75%,470.000000,24.000000,70.000000,905.000000,47.000000,14.000000,19.000000
max,1600.000000,140.000000,925.000000,3701.000000,252.000000,187.000000,260.000000


In [73]:
data_comp.to_csv('CleanedRestaurant.csv', index = False)